In [1]:
import polars as pl

import rustystats as rs

data = pl.read_parquet("https://raw.githubusercontent.com/PricingFrontier/pricing-data-example/917c853e256df8d5814721ab56f72889a908bb08/data/processed/frequency_set.parquet")

In [2]:
train_data = data.filter(pl.col("Group") != "5")
test_data = data.filter(pl.col("Group") == "5")

In [3]:
#creates analysis/exploration.json
exploration = rs.explore_data(
    data=train_data,
    response="ClaimCount",
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ],
    exposure="Exposure",
)

In [4]:
# CV-based regularization selection (recommended)
result = rs.glm(
    "ClaimCount ~ VehAge + BonusMalus + TE(Region)", 
    data, 
    family="negbinomial", 
    offset="Exposure").fit(
    regularization="elastic_net",
)

print(f"Selected alpha: {result.alpha}")
print(f"CV deviance: {result.cv_deviance}")

Selected alpha: 1e-06
CV deviance: 0.2757213187955567


In [5]:
print(result.summary())

                                 GLM Results                                  

Family:              NegativeBinomial(theta=1.5420) No. Observations:        678012
Link Function:       (default)       Df Residuals:            678008
Method:              IRLS + Ridge    Df Model:                     3
Scale:               0.2873          Alpha (λ):               0.0000
L1 Ratio:            0.00            Iterations:                  10
Non-zero coefs:      3              

Log-Likelihood:         -141369.1430 Deviance:                194792.8018
AIC:                     282746.2860 Null Deviance:            70452.7426
BIC:                     282791.9937 Pearson chi2:             1663586.35
Converged:           True           

------------------------------------------------------------------------------
Variable               Coef    Std.Err        z    P>|z|                 95% CI     
------------------------------------------------------------------------------
Intercept          

In [6]:
#writes analysis/diagnostics
result.diagnostics(
    train_data=train_data,
    test_data = test_data,
    categorical_factors=["Region", "Area", "VehBrand", "VehGas"],
    continuous_factors=["VehPower", "VehAge", "DrivAge", "BonusMalus" ]
)

ModelDiagnostics(model_summary={'formula': 'ClaimCount ~ VehAge + BonusMalus + TE(Region)', 'family': 'NegativeBinomial(theta=1.5420)', 'link': 'log', 'n_obs': 542055, 'n_params': 4, 'df_resid': 542051, 'converged': True, 'iterations': 10, 'regularization': {'type': 'none', 'alpha': 1e-06, 'l1_ratio': 0.0, 'cv_deviance': np.float64(0.275721), 'cv_folds': 5, 'selection': 'min'}}, fit_statistics={'deviance': 194792.8018223345, 'null_deviance': 70452.74258489985, 'deviance_explained': -1.7648718087531834, 'log_likelihood': -112871.16896538173, 'aic': 225750.33793076346, 'bic': 225795.1504217686, 'pearson_chi2': 1320521.7285009443, 'dispersion': 2.4361577204007454}, loss_metrics={'family_deviance_loss': 0.28674455390709597, 'mse': 0.05423527567790984, 'mae': 0.09773522268691066, 'rmse': 0.23288468321877642}, calibration={'ae_ratio': 0.994, 'hl_pvalue': 0.0, 'problem_deciles': [{'decile': 0, 'ae': 2.32, 'n': 196847, 'ae_ci': [2.26, 2.38]}, {'decile': 1, 'ae': 1.19, 'n': 64191, 'ae_ci': [1.1